In [168]:
%reload_ext autoreload
%autoreload 2

import os

import yaml

import basedosdados as bd
import pandas as pd

from pathlib import Path

- identificar dataset e table ID
- sync_bucket -> public_bucket= basedosdados-dev to basedosdados
- tablecreate(), table.publish()

In [2]:
!pwd

/mnt/AEB0CCA7B0CC777D/Jlab/gabinete_sv/bd+/mais_projects/notebooks


In [185]:
from basedosdados import Storage
def sync_bucket(
    source_bucket_name,
    dataset_id,
    table_id,
    destination_bucket_name,
    backup_bucket_name,
    mode="staging",
):
    """Copies proprosed data between storage buckets.
    Creates a backup of old data, then delete it and copies new data into the destination bucket.

    Args:
        source_bucket_name (str):
            The bucket name from which to copy data.
        dataset_id (str):
            Dataset id available in basedosdados. It should always come with table_id.
        table_id (str):
            Table id available in basedosdados.dataset_id.
            It should always come with dataset_id.
        destination_bucket_name (str):
            The bucket name which data will be copied to.
            If None, defaults to the bucket initialized when instantianting Storage object
            (check it with the Storage.bucket proprerty)
        backup_bucket_name (str):
            The bucket name for where backup data will be stored.
        mode (str): Optional.
         Folder of which dataset to update.

    Raises:
        ValueError:
            If there are no files corresponding to the given dataset_id and table_id on the source bucket
    """

    ref = Storage(dataset_id=dataset_id, table_id=table_id)

    prefix = f"{mode}/{dataset_id}/{table_id}/"

    source_ref = (
        ref.client["storage_staging"]
        .bucket(source_bucket_name)
        .list_blobs(prefix=prefix)
    )

    destination_ref = ref.bucket.list_blobs(prefix=prefix)

    if len(list(source_ref)) == 0:

        raise ValueError("No objects found on the source bucket")

    else:
        # MAKE A BACKUP OF OLD DATA
        if len(list(destination_ref)):
            ref.copy_table(
                source_bucket_name=destination_bucket_name,
                destination_bucket_name=backup_bucket_name,
            )

            # DELETE OLD DATA FROM PROD
            ref.delete_table(not_found_ok=True)

        # COPIES DATA TO DESTINATION
        ref.copy_table(source_bucket_name=source_bucket_name)


In [190]:
from basedosdados.base import Base
import basedosdados as bd

def load_configs(dataset_id, table_id):
    configs_path = Base()._load_config()
    metadata_path = configs_path["metadata_path"]
    table_path = f"{metadata_path}/{dataset_id}/{table_id}"
    
    return yaml.load(
        open(f"{table_path}/table_config.yaml", "r"), Loader=yaml.FullLoader
    ), configs_path


def replace_project_id(configs_path):
    
    bq_prod_id = configs_path['gcloud-projects']['prod']['name']
    bq_staging_id = configs_path['gcloud-projects']['staging']['name']
    
    sql_file = Path(table_path  + '/publish.sql').open('r').readlines()

    sql_lines = [line for line in sql_file if f'.{table_id}' in line]

    prod_id    = sql_lines[0].split('VIEW ')[1].split('.')[0]
    staging_id = sql_lines[1].split('from ')[1].split('.')[0]

    sql_final = [line.replace(f'VIEW {prod_id}',f'VIEW {bq_prod_id}') for line in sql_file]
    sql_final = [line.replace(f'from {staging_id}',f'from {bq_staging_id}') for line in sql_final]
    
    
    Path(table_path  + '/publish.sql').open('w').write(''.join(sql_final))
    

def is_partitioned(table_config):
    return table_config["partitions"] is not None


def push_table_to_bq(dataset_id,
                     table_id,
                     source_bucket_name='basedosdados',
                     destination_bucket_name='basedosdados-sv',
                     backup_bucket_name='basedosdados-sv-bkp'
                    ):
    
    sync_bucket(source_bucket_name,
            dataset_id,
            table_id,
            destination_bucket_name,
            backup_bucket_name
        )
    
    table_config, configs_path = load_configs(dataset_id, table_id)
    replace_project_id(configs_path)
    
    tb = bd.Table(table_id, dataset_id)
    tb.create('',
          if_table_exists="replace",
          if_storage_data_exists="pass",
          if_table_config_exists="pass",
          partitioned=is_partitioned(table_config))
    
    tb.publish(if_exists="replace")
    

In [203]:
push_table_to_bq(dataset_id='a_proposal_test', table_id='proposal_test_table')

staging/a_proposal_test/proposal_test_table/test_data.csv copied sucessfully to basedosdados-sv


Uploading files:   0%|          | 0/1 [00:00<?, ?it/s]

gabinete-sv gabinete-sv


Uploading files: 100%|██████████| 1/1 [00:00<00:00,  2.21it/s]


In [199]:
tb = bd.Table('proposal_test_table','a_proposal_test')

In [201]:
tb.delete('all')

In [202]:
bd.Storage('a_proposal_test','proposal_test_table').delete_table()

staging/a_proposal_test/proposal_test_table/test_data.csv deleted sucessfully!
